In [1]:
import win32com.client
import time
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits

In [2]:

# Launch ASCOM chooser
chooser = win32com.client.Dispatch("ASCOM.Utilities.Chooser")
chooser.DeviceType = "Camera"  # Can be 'Telescope', 'Camera', 'FilterWheel', etc.

# Pick ZWO ASCOM driver (GUI opens)
camera_id = chooser.Choose(None)

print("You selected:", camera_id)

You selected: ASCOM.ASICamera2.Camera


In [3]:
camera = win32com.client.Dispatch("ASCOM.ASICamera2.Camera")
camera.Connected = True
print("Connected:", camera.Connected)
cam_width = camera.CameraXSize
cam_height = camera.CameraYSize

# Setup camera for exposure
camera.BinX = 1
camera.BinY = 1
camera.NumX = 1000
camera.NumY = 1000
camera.StartX = cam_width // 2 - 500
camera.StartY = cam_height // 2 - 500

# Start an exposure (e.g. 1 second, false = light frame)
exposure_duration = 0.001
camera.StartExposure(exposure_duration, False)

# Wait for exposure to finish
while not camera.ImageReady:
    time.sleep(0.1)

# Get image data
image_data = np.array(camera.ImageArray)  # This will be a 2D array of pixel values

# Done
camera.Connected = False
print("Image captured and camera disconnected.")

Connected: True
Image captured and camera disconnected.


In [4]:
print(image_data)

[[65535 65535 65535 ... 65535 65535 65535]
 [65535 65535 65535 ... 65535 65535 65535]
 [65535 65535 65535 ... 65535 65535 65535]
 ...
 [65535 65535 65535 ... 65535 65535 65535]
 [65535 65535 65535 ... 65535 65535 65535]
 [65535 65535 65535 ... 65535 65535 65535]]
